In [1]:
import os
import pandas as pd
import ast
import re

from gensim.corpora.dictionary import Dictionary

In [2]:
df = pd.read_csv('collection_test.csv')
df.head()

,filename,title,author,pub_date,pub_place,tokens,types,characters,text,corpus
0,343012,Christliche Leich- vnd Ehren-Predigt bey der S...,Martinus Seidemannus,1625,Görlitz,3719,1521,24792,"['', 'Chriſtliche', 'Leich', 'vnd', 'EhrenPred...",['aedit']
1,343014,THRENOLOGIA. Christliche Leich vnd EhrenSermon...,Tobias Seiler,1605,Leipzig,10097,3476,73098,"['', 'THRENOLOGIA', 'Chriſtliche', 'Leich', 'v...",['aedit']
2,343016,Kurtze Anleitung: Wie die jetzige böse Zeit/ d...,Christoph von Reideburg,1642,Breslau,11515,3440,78795,"['', 'Kurtze', 'Anleitung', 'Wie', 'die', 'jet...",['aedit']
3,343019,"mors beatorvm EXODUS MISERIÆ TERRESTRIS, ET PA...",Christian Adolph,1636,Leipzig,7611,2546,50565,"['', 'mors', 'beatorvm', 'EXODUS', 'MISERIÆ', ...",['aedit']
4,345804,Geistlich Paßport Oder Richtige Kundschafft de...,Leonhard Felber,1606,Oels,11482,3186,75172,"['', 'Geiſtlich', 'Paſsport', 'Oder', 'Richtig...",['aedit']


In [3]:
df['text'] = df['text'].apply(lambda x: ast.literal_eval(x))

In [4]:
import spacy

In [5]:
nlp = spacy.load('de_dep_news_trf')

In [6]:
stopwords = list(nlp.Defaults.stop_words)#+['deß', 'vnnd', 'unnd', 'jhme', 'inn', 'unter', 'vnd', 'bey', 'jetzo', 'jetzto', 'alß', 'uber', 'nit', 'alß',
                                            #'unſer', 'ꝟ', 'vnsers','vnserm', 'vnsere', 'ſoll', 'ut', 'wirdt', 'umb', 'nunc', 'qui', 'que', 'quod', 'ac', 'hoc']

In [7]:
# def clean_tokens(tokens):
#     for i, token in enumerate(tokens):
#         tokens[i] = re.sub(r'ꝛ', 'z', token)
#         tokens[i] = re.sub(r'ſ', 's', token)
#     return tokens

In [9]:
# df['text_clean'] = df['text'].apply(lambda x: [string.replace('ꝛ', 'z').replace('ſ', 's') for string in x])
df['text'] = df['text'].apply(lambda x: [t.lower() for t in x if t.lower() not in stopwords])

In [10]:
dictionary = Dictionary(df['text'])
dictionary.filter_extremes(no_below=3, no_above=0.5)
l = list(dictionary.items())
print(len(l))
print(l)

50071
[(0, '119'), (1, '125'), (2, '1585'), (3, '1602'), (4, '1625'), (5, '28'), (6, '2maria'), (7, '30'), (8, '31'), (9, '38'), (10, '39'), (11, '40'), (12, '46'), (13, '71'), (14, '91'), (15, '94'), (16, 'abgelaſſen'), (17, 'abgeleſene'), (18, 'abgematteten'), (19, 'abgetrieben'), (20, 'abunda'), (21, 'abzuſcheiden'), (22, 'abſondern'), (23, 'ac'), (24, 'achtung'), (25, 'adam'), (26, 'adverſitate'), (27, 'agone'), (28, 'alhier'), (29, 'allhie'), (30, 'allhier'), (31, 'alzu'), (32, 'amiſiſti'), (33, 'anbricht'), (34, 'andacht'), (35, 'anfechtung'), (36, 'anfechtungen'), (37, 'angefochten'), (38, 'angehoret'), (39, 'angenehmes'), (40, 'angethan'), (41, 'anklager'), (42, 'anklebende'), (43, 'ankunfft'), (44, 'anlangend'), (45, 'anmuttiger'), (46, 'ans'), (47, 'antlitz'), (48, 'antworten'), (49, 'apoſtel'), (50, 'appliciren'), (51, 'arbei'), (52, 'arme'), (53, 'aufferziehen'), (54, 'aufferzogen'), (55, 'aufferſtehug'), (56, 'aufferſtehung'), (57, 'auffgerichtet'), (58, 'auffs'), (59, 'au

In [11]:
corpus = [dictionary.doc2bow(a) for a in df['text']]

In [12]:
from gensim.models import LdaMulticore

In [13]:
max_topics = 50
    
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=50, workers = 12)

In [ ]:
lda_model.print_topics(num_words=10, num_topics=max_topics)

[(0,
  '0.003*"vnnd" + 0.003*"leichpredigt" + 0.002*"ꝟ" + 0.002*"uber" + 0.001*"unſer" + 0.001*"ſoll"'),
 (1,
  '0.009*"vnnd" + 0.003*"ꝟ" + 0.002*"leichpredigt" + 0.001*"herꝛn" + 0.001*"vnſere" + 0.001*"ſoll"'),
 (2,
  '0.003*"vnnd" + 0.002*"uber" + 0.002*"worte" + 0.002*"leichpredigt" + 0.001*"gerechten" + 0.001*"p"'),
 (3,
  '0.024*"vnnd" + 0.004*"leichpredigt" + 0.002*"ꝟ" + 0.002*"ut" + 0.002*"inn" + 0.002*"qui"'),
 (4,
  '0.004*"vnnd" + 0.002*"qui" + 0.002*"ut" + 0.002*"uber" + 0.002*"p" + 0.002*"leichpredigt"'),
 (5,
  '0.008*"vnnd" + 0.002*"leichpredigt" + 0.002*"uber" + 0.001*"qui" + 0.001*"herꝛ" + 0.001*"nit"'),
 (6,
  '0.013*"vnnd" + 0.004*"ꝟ" + 0.003*"leichpredigt" + 0.001*"qui" + 0.001*"nit" + 0.001*"vnſere"'),
 (7,
  '0.007*"vnnd" + 0.003*"leichpredigt" + 0.002*"unnd" + 0.002*"ꝟ" + 0.001*"vnſere" + 0.001*"ut"'),
 (8,
  '0.008*"vnnd" + 0.002*"qui" + 0.002*"inn" + 0.002*"leichpredigt" + 0.002*"ꝟ" + 0.001*"ut"'),
 (9,
  '0.004*"leichpredigt" + 0.002*"vnnd" + 0.002*"uber" + 0.0